<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> HomeWork 1 | Question 3
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# PySpark Setting

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
spark

# Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from pyspark.sql.types import FloatType, ArrayType, IntegerType
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql import Window

# Dataset

In [6]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 167MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
replace Bigdata_hw2_datasets/q1/stream_data_dgim.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [7]:
dataset_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/data.txt", header=None, names=["data"])
dataset_pandas["data"] = dataset_pandas["data"].apply(lambda x:x.split(" "))
dataset_pandas["data"] = dataset_pandas["data"].apply(lambda item: [float(x) for x in item])
dataset_pandas["data-id"] = range(len(dataset_pandas))

c1_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/c1.txt", header=None, names=["center"])
c1_pandas["center"] = c1_pandas["center"].apply(lambda x:x.split(" "))
c1_pandas["center"] = c1_pandas["center"].apply(lambda item: [float(x) for x in item])
c1_pandas["center-id"] = range(len(c1_pandas))

c2_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/c2.txt", header=None, names=["center"])
c2_pandas["center"] = c2_pandas["center"].apply(lambda x:x.split(" "))
c2_pandas["center"] = c2_pandas["center"].apply(lambda item: [float(x) for x in item])
c2_pandas["center-id"] = range(len(c2_pandas))

In [8]:
NUMBER_FEATURES = len(dataset_pandas["data"].values[0])

# KMeans

In [9]:
class SparkKMeans():
  def __init__(self, dataset, center, description, distance_metric_name):
    self.dataset_spark = spark.createDataFrame(dataset)
    self.center_spark = spark.createDataFrame(center)
    self.description = description

    if distance_metric_name == "Euclidean":
      self.distance_metric = self.euclidean_distance
    elif distance_metric_name == "Manhatan":
      self.distance_metric = self.manhatan_distance
    else:
      raise Exception("Invalid distance_metric_name!")

  def run(self, number_steps):
    self.number_steps = number_steps
    self.costs = []
    for step in range(number_steps):
      data_center_spark = self.dataset_spark.join(self.center_spark)\
                              .withColumn('distance', self.distance_metric(self.dataset_spark.data, self.center_spark.center))

      data_center_spark = data_center_spark.withColumn("min-distance", f.min("distance")\
                                           .over(Window.partitionBy("data-id")))\
                                           .where(f.col("min-distance") == f.col("distance"))\
                                           .drop(*["min-distance", "center"])


      step_cost = data_center_spark.withColumn("sum-distance", f.col("distance")**2)\
                                   .select("sum-distance")\
                                   .groupBy().sum().collect()[0][0]

      self.costs.append(step_cost)
      print(f"Step = {step}, Cost = {step_cost}")

      data_center_mapping = data_center_spark.drop(*["data", "distance"])\
                                             .toDF("data-id", "center-id")\
                                             .toPandas()
      
      data_center_spark = data_center_spark.groupBy("center-id")\
                                           .agg(f.collect_list("data")\
                                           .alias("data-list"))

      array_mean_udf = udf(self.array_mean, ArrayType(FloatType()))

      self.center_spark = data_center_spark.select("center-id", array_mean_udf("data-list")\
                                           .alias("center"))

    return data_center_mapping

  def report(self):
    print(f"Change cost percent in ten steps {int((self.costs[0] - self.costs[10]) / self.costs[0] * 100)}%\n") 

    plt.scatter(range(1, self.number_steps+1), self.costs)
    plt.plot(range(1, self.number_steps+1), self.costs)
    plt.xlabel("Step")
    plt.ylabel("Cost")
    plt.title(f"Change Cost in KMeans\n{self.description}")
    plt.show()

  @staticmethod
  def euclidean_distance(a, b):
    distance = 0
    for i in range(NUMBER_FEATURES):
      distance += (a[i] - b[i]) ** 2
    return distance ** 0.5

  @staticmethod
  def manhatan_distance(a, b):
    distance = 0
    for i in range(NUMBER_FEATURES):
      distance += f.abs(a[i] - b[i])
    return distance

  @staticmethod
  def array_mean(x):
    number_data = len([i for i in zip(*x)][0])
    sum_array = [sum(i) for i in zip(*x)]
    return [item/number_data for item in sum_array]

# Results

In [ ]:
for distance_metric_name in ["Euclidean", "Manhatan"]:
  for center, center_name in [(c1_pandas, "C1"), (c2_pandas, "C2")]:
    print(f"Distance Metric = {distance_metric_name} | Center Name = {center_name}")

    kmeans = SparkKMeans(dataset=dataset_pandas, center=center,
                         description=f"{center_name}-{distance_metric_name}",
                         distance_metric_name=distance_metric_name)

    kmeans_output = kmeans.run(number_steps=20)

    kmeans.report()    

Distance Metric = Euclidean | Center Name = C1
Step = 0, Cost = 623660345.3064235
Step = 1, Cost = 509862910.53849345
Step = 2, Cost = 485480688.18347406
Step = 3, Cost = 463997009.2097292
Step = 4, Cost = 460969266.6840041
Step = 5, Cost = 460537848.1230785
Step = 6, Cost = 460313099.4867738
Step = 7, Cost = 460003523.95227706
